<a href="https://colab.research.google.com/github/williamlidberg/Analyses-of-Environmental-Data-2/blob/main/modules/module_10/Assignment_10_webmapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creating a web map
Notebooks are great for developing and GIS softwares like arcgis/QGis are nice for exploring data. But your ability to share your work is limited, at best you can print a PDF like a caveman. That is where web mapping comes in. Web mapping brings a level of accessibility and insight that traditional maps simply can't match.

\
Let's start by creating a very basic map. These lines of code will create a basemap and save it as a html file.

In [ ]:
import folium

m = folium.Map()
m.save('/content/map.html')

### Assignment 1
Download the html file to your computer and open it with your browser.

The map is based on OpenStreetMap. Like in previous exercises we can make it look cooler by using darkmode. We can also set a start location and zoom level. Note that the right zoom start level will depend on your screen size. Smaller screens can fit less map.

In [ ]:
m = folium.Map(location = [60.663, 16.885], zoom_start=10) # Gävle, I am doing some science here.
folium.TileLayer('Cartodb dark_matter').add_to(m) # Sets Tile Theme to (Dark Theme)
m.save('/content/cool_map.html')

Now you can add a basemap and select an area where the map should focus. Now we can add some data to the map. This example uses harvest blocks from the Swedish forest agency. Start by downloading the data.

In [ ]:
!wget https://geodpags.skogsstyrelsen.se/geodataport/data/sksAvverkAnm21.zip
!unzip /content/sksAvverkAnm21.zip

You can import the shapefile using geopandas.

In [ ]:
import geopandas as gpd

harvests = gpd.read_file('/content/sksAvverkAnm21.shp')
harvests.head()

Combine the dataframe with folium to create a html map. You can also change settings for the colors of the polygons and thicknes of the lines.

In [ ]:
m = folium.Map(location = [61.526, 16.413], zoom_start=9) # Gävle
folium.TileLayer('cartodbpositron').add_to(m) # Sets Tile Theme to gray

choropleth = folium.Choropleth(
    geo_data=harvests,  # Harvest sites geodataframe
    fill_color='red',
    line_weight=0.1,
    line_color='white',
).add_to(m)

m.save('/content/harvest_map.html')

You can also subsample the data to show the most recent applications.

In [ ]:
recent_harvest_applications = harvests[harvests['ArendeAr'] > 2022]

m = folium.Map(location=[61.345, 16.514], zoom_start=9, tiles='cartodbpositron') # Sets Tile Theme to Gray

folium.Choropleth(
    geo_data=recent_harvest_applications,
    name='Choropleth',
    data=recent_harvest_applications,
    columns=['OBJECTID', 'AnmaldHa'], # OBJECTID is the ID column of the
    key_on="feature.properties.OBJECTID",
    fill_color='YlGnBu',
    fill_opacity=1,
    line_opacity=0.2,
    legend_name='Harvest area in ha',
    smooth_factor=0,
    control=False  # Add this line to hide the default legend
).add_to(m)

# Add legend to the map
folium.LayerControl().add_to(m)

m.save('/content/recent_harvest_map.html')


### Task 2
Make a html file for a webmap from wetlands in Gävleborg
https://geodata.naturvardsverket.se/nedladdning/VMI/ursprunglig_digitalisering/X_Gavleborg_VMI.zip

In [ ]:
!wget https://geodata.naturvardsverket.se/nedladdning/VMI/ursprunglig_digitalisering/X_Gavleborg_VMI.zip
!unzip /content/X_Gavleborg_VMI.zip

# Heatmaps
A popular way to display large amounts of data is to create a heatmap. For this example you will use location of houses in Gävle. The data is avalible at [Sveriges Dataportal](https://www.dataportal.se/datasets/574_1040/baskarta-byggnader#ref=?p=4&q=g%C3%A4vle&s=2&t=20&f=&rt=dataset%24esterms_IndependentDataService%24esterms_ServedByDataService&c=false).




In [ ]:
from urllib.request import urlretrieve
url = ('https://catalog.gavle.se/store/1/resource/48')
filename = '/content/baskarta-adresser.zip' # you need to adjust this path on your own computer if you are using anaconda.
urlretrieve(url, filename)
!unzip /content/baskarta-adresser.zip

In [ ]:
import geopandas as gpd
buildings = gpd.read_file('/content/baskarta_adress.shp')
# Add a new column named 'ID' with sequential IDs starting from 1
buildings['id'] = range(1, len(buildings) + 1)
buildings

In [ ]:
from folium.plugins import HeatMap


m = folium.Map(location=[buildings['WGS84_LAT'].mean(), buildings['WGS84_LONG'].mean()], zoom_start=9)
heat_data = [[row['WGS84_LAT'], row['WGS84_LONG']] for index, row in buildings.iterrows()]
HeatMap(heat_data).add_to(m)

map_title = "Heatmap of buildings"
title_html = f'<h1 style="position:absolute;z-index:100000;left:40vw" >{map_title}</h1>'
m.get_root().html.add_child(folium.Element(title_html))
# Save the map
m.save('/content/index.html') # note that I renamed it index.html for this to work with github.


Zoom in and out on the heatmap and watch how it dynamically aggregates the data.


## Task 3 Publish the web map.. on the web
So far we have created a html file that can be opened in a browser. While this file can be shared it is not yet a webmap. For that we need a way to host it online. There are a few different way of doing that. There are multiple services which can host html files for you or you can host them yourself from a computer or server. Read the instructions bellow and pick a way to host your html file to the world. The GitHub option is easier and will be enough for you to pass the assignment. The other alternative is for you to explore if you like.

Include the URL somewhere in the notebook so I can have a look at your webmap.

## From GitHub
For this option you need to create a GiHub account. GitHub is a developer tool that can be used to devlop and share projects. Once you have created a github account you can create follow these [instructions](https://docs.github.com/en/pages/quickstart) to create a new repository and activate github pages. The key parts of the instructions are:


1.   Create a new repository with default settings and name it username.github.io where username is your github username. for me this would be williamlidberg.github.io
3.   On the next page you will be given the option to create a new file or upload an existing file. Upload the heatmap html file created above.
2.   Wait 10 minutes and navigate to username.github.io with username replaced with your own. For me this would be https://williamlidberg.github.io/ it will display error 404 if you dont wait a bit.

## From a server

This option requires a bit more effort since you need to have a computer running all the time to host the page (or at least when you want it to be available). You will also need to open up some ports in your routers settings for the world to access your page. I will not ask you to do this for this course but you can use a server that I have configured for you.

### If you want to place the web map on the server I have set up

Keep in mind that for this alternative I will clean out your files after the course ends, so the other alternatives are better if you want to keep them after (you can of course upload your files anywhere in the future as well). Also keep in mind that everyone else will have full access to your files on this server, **do not upload anything you dont want everyone to see**.

Download and install FileZilla client, it is a free file transfer application available for all major platforms. You can find it here: https://filezilla-project.org/download.php?type=client

Launch the application and then fill in the fields at the top of the window as follows:

Host: webmaps.konvergens.se<br>
Username: student<br>
Password: course_slu2024<br>
Port: 21 (you can also leave this blank)<br>

Then press Quickconnect. Some status messages should scroll by and then you will be connected.

On the left hand side of the main window are your local files, and on the right hand side are the files on the server. You can move files between the server and your computer by either dragging and dropping, or double-clicking the file. Create a folder on the server by right-clicking the file listing on the server and selecting "Create directory", give the directory a memorable name, your student login name for example. Now navigate to the new folder and upload your HTML-file.

After uploading, you will be able to visit your webmap in a browser by entering the following in the address field (replace foldername and filename.html as necessary to reflect the folder you created and the name of your html-file):

http://webmaps.konvergens.se/foldername/filename.html

You can also just visit http://webmaps.konvergens.se/ and navigate to your file by clicking the links.

### If you want to try hosting your own server

There are several options for web server software available, depending on your operating system. One of the popular options is Apache HTTPD, which is available for all major platforms, another is Microsoft IIS which is available for Windows operating systems. However, the rising star and also my personal favorite is NGINX (pronounced Engine X), which is available for all major platforms and several smaller platforms.

The installation instructions will vary slightly depending on your operating system so I have divided them up based on your OS below. The default configuration should work perfectly for simple sites such as these.

For all operating systems, after nginx is up and running your site will be visible locally to you if you visit http://localhost in your web browser (http://localhost:8080 on MacOS), others will have to enter your public IP-address in the address field of their browser instead of localhost, you can find your public IP by visiting i.a. https://whatismyip.org/ (make sure you are not connected to a VPN), this IP may change if you restart your router. If you get a "secure connection failed" error, make sure that the browser has not replaced the http:// with http**s**://.

Its important to note that there are other things to keep in mind if you plan on hosting your own web server in the future. Proper configuration and keeping the operating system and web server software up-to date is absolutely crutial to limit the risk of malevolent individuals breaching your server and using it for their own purposes. In these cases, you should read up more closely on security best practices and how to enable HTTPS for your website (which is almost a must for any serious website these days).

#### Windows

Download the latest release of NGINX from https://nginx.org/en/download.html (https://nginx.org/download/nginx-1.26.0.zip at the time of writing) and extract the downloaded .zip-file.

Within the extracted directory, you should see an executable named nginx.exe and a couple of directories. The *conf* directory contains the configuration files and the ***html*** directory contains the files that will be visible to those visiting your server in a web browser.

Start nginx by navigating to the extracted folder in a terminal and typing: start nginx.

Windows should alert you that NGINX wants to communicate with the internet, press allow to let it through the firewall.

#### Linux

Nginx is almost certainly available in your distributions repositories, so you should install it from there.

1. **Debian/Ubuntu/Linux Mint/PopOS:** *sudo apt install nginx*
2. **RedHat/Fedora/CentOS/Rocky/Alma:** *sudo dnf in nginx*
3. **Arch/Manjaro:** *sudo pacman -S nginx*
4. **openSUSE:** *sudo zypper in nginx*

After installation, enable it using: *sudo systemctl enable --now nginx*.

The default configuration might vary slightly between distributions, but in most cases you will want to place your html files in the ***/var/www/html***-folder.

If you have a firewall installed, you will have to allow connections over HTTP, this will vary slightly depending on your firewall software.

* **For ufw (default on ubuntu):** *sudo ufw allow http*
* **For firewalld (default on RedHat and Fedora):** *sudo firewall-cmd --permanent --zone=public --add-service=http*, followed by *sudo firewall-cmd --reload*

#### MacOS

The easiest way to install NGINX on mac is by using homebrew. If you dont have homebrew installed you can follow the instructions here to get it: https://docs.brew.sh/Installation

Then install by typing (in the terminal): *brew install nginx*, it will take a while to complete.

Now you can enable nginx by typing: *brew services start nginx*.

You will want to place your html files in the ***/opt/homebrew/var/www***-folder

If you want to enable connections from outside your home network, you need to let them through the firewall. You can do that by executing (in the terminal): *sudo /usr/libexec/ApplicationFirewall/socketfilterfw --add /usr/local/bin/nginx*

#### Opening up your web server to the outside world

If you set up your own web server, you will probably need to also open the correct port in your router. This is not necessary if you are not using a router (i.e. the computer is connected via cable directly to the wall or fiber converter). The instructions in this case will vary widely depending on the make and model of your router, so it is not feasible to provide detailed instructions here. However, the basic steps are:

1. Assign a static internal IP-address to the computer that is running your web server.
2. Forward port 80 (or port 8080 for MacOS) to the static IP you set in the first step.

Port forwarding is explained here for Asus routers: https://www.asus.com/support/faq/1037906/, and here for TP-Link routers: https://www.tp-link.com/se/support/faq/1379/. These are probably the two most common consumer-grade routers.